# Marketing Maturity Analysis

This notebook loads survey data from Google Sheets and analyzes marketing maturity scores.


In [56]:
# Import required packages
import pandas as pd
import numpy as np
import requests
import io
import urllib3

# Suppress SSL warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

print("✓ Packages imported")


✓ Packages imported


In [57]:
# Google Sheet configuration
SHEET_ID = '1tHWUJWJl_zTwGRTg21qbW_5qpYH8bBMFYZYbIZ03eO8'
GID = '491555971'
SHEET_URL = f'https://docs.google.com/spreadsheets/d/{SHEET_ID}/export?format=csv&gid={GID}'

print(f"Sheet URL configured")


Sheet URL configured


In [58]:
# Load data from Google Sheet
response = requests.get(SHEET_URL, verify=False, timeout=30)
response.raise_for_status()
df = pd.read_csv(io.StringIO(response.text))

print(f"✓ Data loaded: {df.shape[0]} rows, {df.shape[1]} columns")


✓ Data loaded: 3 rows, 26 columns


In [59]:
# Display the data
df.head()


,Timestamp,Email Address,How would you rank the capabilities of your existing CRM technology stack?,How would you rank the utilisation of your existing CRM technology stack?,How integrated are the tools in your CRM tech stack?,How much data are you capturing on your customers?,To what extent is all of your customer data in one place and accessible to the marketing team?,Do you have a bank of channel-specific templates and creative assets?,To what degree are campaigns planned and executed across channels?,Are you speaking to your customers at key lifecycle moments?,...,To what degree are your insights automatically available?,Are test learnings documented and used to inform decision-making?,"What method of performance attribution are you using, and is it applied consistently?",What raw data do you have available for insights?,How much do you know about your customers?,Do you have a view of the behavioural impact of your campaigns?,How well-defined are CRM roles and responsibilities within your team?,What level of CRM expertise exists across your organisation?,How embedded are CRM processes in your day-to-day operations?,How would you rate your teamâs ability to launch CRM campaigns quickly and accurately?
0,11/2/2025 17:13:08,louise.trelles-tvede@indigital.marketing,2,1,1,1,1,1,1,2,...,1,1,1,1,1,1,3,1,1,1
1,11/4/2025 14:48:53,louisettvede@gmail.com,4,4,3,2,1,2,4,2,...,1,1,2,1,1,2,3,1,1,1
2,11/7/2025 10:00:23,shahruzahmed1995@gmail.com,3,2,3,3,3,4,2,3,...,3,2,3,2,3,2,2,3,2,2


## Clean Column Names & Question Categories

First, we'll clean the column names (lowercase, remove punctuation) for easier mapping, then define categories.


In [60]:
# Function to clean column names
import re

def clean_column_name(col_name):
    """Clean column name: lowercase, remove ALL special characters, replace spaces with underscores"""
    # Convert to lowercase
    cleaned = col_name.lower()
    # Replace hyphens with spaces (so words don't run together)
    cleaned = cleaned.replace('-', ' ')
    # Remove ALL other special characters (keep only letters, numbers, and spaces)
    cleaned = re.sub(r'[^a-z0-9\s]', '', cleaned)
    # Replace multiple spaces with single space
    cleaned = re.sub(r'\s+', ' ', cleaned)
    # Replace spaces with underscores
    cleaned = cleaned.strip().replace(' ', '_')
    return cleaned

# Create mapping: original -> cleaned
COLUMN_NAME_MAPPING = {}
CLEANED_TO_ORIGINAL = {}

for col in df.columns:
    cleaned = clean_column_name(col)
    COLUMN_NAME_MAPPING[col] = cleaned
    CLEANED_TO_ORIGINAL[cleaned] = col

print(f"✓ Cleaned {len(COLUMN_NAME_MAPPING)} column names")
print(f"\nSample mappings:")
for i, (original, cleaned) in enumerate(list(COLUMN_NAME_MAPPING.items())[:3]):
    print(f"  '{original}' → '{cleaned}'")


✓ Cleaned 26 column names

Sample mappings:
  'Timestamp' → 'timestamp'
  'Email Address' → 'email_address'
  'How would you rank the capabilities of your existing CRM technology stack?' → 'how_would_you_rank_the_capabilities_of_your_existing_crm_technology_stack'


In [61]:
# Display all cleaned question names for reference
question_columns = [col for col in df.columns if col not in ['Timestamp', 'Email Address']]
print(f"All cleaned question names ({len(question_columns)} questions):\n")
for i, q in enumerate(question_columns, 1):
    cleaned = COLUMN_NAME_MAPPING[q]
    print(f"{i:2d}. {cleaned}")


All cleaned question names (24 questions):

 1. how_would_you_rank_the_capabilities_of_your_existing_crm_technology_stack
 2. how_would_you_rank_the_utilisation_of_your_existing_crm_technology_stack
 3. how_integrated_are_the_tools_in_your_crm_tech_stack
 4. how_much_data_are_you_capturing_on_your_customers
 5. to_what_extent_is_all_of_your_customer_data_in_one_place_and_accessible_to_the_marketing_team
 6. do_you_have_a_bank_of_channel_specific_templates_and_creative_assets
 7. to_what_degree_are_campaigns_planned_and_executed_across_channels
 8. are_you_speaking_to_your_customers_at_key_lifecycle_moments
 9. are_you_leveraging_different_send_types
10. how_are_you_managing_the_cadence_of_your_communications
11. to_what_extent_are_you_testing_to_optimise_your_communications
12. to_what_extent_do_you_segment_your_audience_for_targeting
13. to_what_extent_are_you_using_personalisation
14. how_accessible_are_your_segments_for_activation
15. to_what_degree_are_your_insights_automatically_a

In [62]:
# Get all question columns in order and their cleaned names
question_columns = [col for col in df.columns if col not in ['Timestamp', 'Email Address']]
cleaned_questions = [COLUMN_NAME_MAPPING[q] for q in question_columns]

# Define question categories based on your mapping
# Tech & Data = first 5 questions
# Campaigning & Assets = next 6 questions
# Segmentation & Personalisation = next 3 questions
# Reporting & Insights = next 6 questions
# People & Operations = last 4 questions

QUESTION_CATEGORIES = {
    'Tech & Data': cleaned_questions[0:5],
    'Campaigning & Assets': cleaned_questions[5:11],
    'Segmentation & Personalisation': cleaned_questions[11:14],
    'Reporting & Insights': cleaned_questions[14:20],
    'People & Operations': cleaned_questions[20:24]
}

# Verify the mapping uses actual cleaned names from the data
print("✓ QUESTION_CATEGORIES created using actual cleaned column names from dataframe")
print(f"Total questions mapped: {sum(len(q) for q in QUESTION_CATEGORIES.values())}")

# Create reverse mapping: cleaned_question -> category
QUESTION_TO_CATEGORY = {}
for category, questions in QUESTION_CATEGORIES.items():
    for question in questions:
        QUESTION_TO_CATEGORY[question] = category

print(f"✓ Defined {len(QUESTION_CATEGORIES)} categories")
print(f"✓ Mapped {len(QUESTION_TO_CATEGORY)} questions to categories")
print(f"\nCategories:")
for category, questions in QUESTION_CATEGORIES.items():
    print(f"  - {category}: {len(questions)} questions")


✓ QUESTION_CATEGORIES created using actual cleaned column names from dataframe
Total questions mapped: 24
✓ Defined 5 categories
✓ Mapped 24 questions to categories

Categories:
  - Tech & Data: 5 questions
  - Campaigning & Assets: 6 questions
  - Segmentation & Personalisation: 3 questions
  - Reporting & Insights: 6 questions
  - People & Operations: 4 questions


In [63]:
# Create reverse mapping: cleaned_question -> category
QUESTION_TO_CATEGORY = {}
for category, questions in QUESTION_CATEGORIES.items():
    for question in questions:
        QUESTION_TO_CATEGORY[question] = category

# Verify all questions are mapped
question_columns = [col for col in df.columns if col not in ['Timestamp', 'Email Address']]
cleaned_questions = [COLUMN_NAME_MAPPING[q] for q in question_columns]
unmapped = [q for q in cleaned_questions if q not in QUESTION_TO_CATEGORY]

if unmapped:
    print(f"⚠️  Warning: {len(unmapped)} unmapped questions:")
    for q in unmapped:
        original = CLEANED_TO_ORIGINAL.get(q, q)
        print(f"  - {original} → '{q}'")
else:
    print("✓ All questions are mapped to categories!")
    
print(f"\n✓ Defined {len(QUESTION_CATEGORIES)} categories")
print(f"✓ Mapped {len(QUESTION_TO_CATEGORY)} questions to categories")
print(f"\nCategories and question counts:")
for category, questions in QUESTION_CATEGORIES.items():
    print(f"  - {category}: {len(questions)} questions")
    
# Show the mapping
print(f"\nDetailed mapping:")
for category, questions in QUESTION_CATEGORIES.items():
    print(f"\n{category}:")
    for i, q in enumerate(questions, 1):
        original = CLEANED_TO_ORIGINAL.get(q, q)
        print(f"  {i}. {q}")


✓ All questions are mapped to categories!

✓ Defined 5 categories
✓ Mapped 24 questions to categories

Categories and question counts:
  - Tech & Data: 5 questions
  - Campaigning & Assets: 6 questions
  - Segmentation & Personalisation: 3 questions
  - Reporting & Insights: 6 questions
  - People & Operations: 4 questions

Detailed mapping:

Tech & Data:
  1. how_would_you_rank_the_capabilities_of_your_existing_crm_technology_stack
  2. how_would_you_rank_the_utilisation_of_your_existing_crm_technology_stack
  3. how_integrated_are_the_tools_in_your_crm_tech_stack
  4. how_much_data_are_you_capturing_on_your_customers
  5. to_what_extent_is_all_of_your_customer_data_in_one_place_and_accessible_to_the_marketing_team

Campaigning & Assets:
  1. do_you_have_a_bank_of_channel_specific_templates_and_creative_assets
  2. to_what_degree_are_campaigns_planned_and_executed_across_channels
  3. are_you_speaking_to_your_customers_at_key_lifecycle_moments
  4. are_you_leveraging_different_send_ty

## Calculate Category Scores

For each client (email address), calculate their average score for each topic/category.


In [64]:
# Create reverse mapping for faster lookup (cleaned -> original)
# Make sure we use the same cleaned names that are in QUESTION_CATEGORIES
CLEANED_TO_ORIGINAL_COL = {cleaned: original for original, cleaned in COLUMN_NAME_MAPPING.items()}

# Verify all cleaned names in QUESTION_CATEGORIES exist in the mapping
all_cleaned_in_categories = []
for questions in QUESTION_CATEGORIES.values():
    all_cleaned_in_categories.extend(questions)

missing = [q for q in all_cleaned_in_categories if q not in CLEANED_TO_ORIGINAL_COL]
if missing:
    print(f"⚠️  Warning: {len(missing)} cleaned names not found in mapping:")
    for q in missing:
        print(f"  - {q}")
else:
    print(f"✓ All {len(all_cleaned_in_categories)} cleaned question names found in mapping")

# Function to calculate category scores for a single client
def calculate_category_scores(client_row, question_categories, cleaned_to_original):
    """
    Calculate average score for each category for a single client
    
    Parameters:
    - client_row: pandas Series (one row from df)
    - question_categories: dict mapping category -> list of cleaned question names
    - cleaned_to_original: dict mapping cleaned column -> original column name
    
    Returns:
    - dict with category -> average score
    """
    scores = {}
    
    for category, questions in question_categories.items():
        category_scores = []
        
        for cleaned_q in questions:
            # Get the original column name
            original_col = cleaned_to_original.get(cleaned_q)
            
            if original_col:
                # Try to get value using .get() with default None
                value = client_row.get(original_col, None)
                
                # Only include if it's a valid number (1-4)
                if value is not None and pd.notna(value):
                    try:
                        num_value = float(value)
                        if 1 <= num_value <= 4:  # Valid maturity score range
                            category_scores.append(num_value)
                    except (ValueError, TypeError):
                        pass  # Skip invalid values
        
        # Calculate average for this category
        if category_scores:
            scores[category] = np.mean(category_scores)
        else:
            scores[category] = None  # No valid scores found
    
    return scores

# Test on first client
print("Testing on first client:")
first_client = df.iloc[0]
first_scores = calculate_category_scores(first_client, QUESTION_CATEGORIES, CLEANED_TO_ORIGINAL_COL)
print(f"Email: {first_client['Email Address']}")
print("\nCategory Scores:")
for category, score in first_scores.items():
    if score is not None:
        print(f"  {category}: {score:.2f}")
    else:
        print(f"  {category}: No valid scores")

# Quick verification - check first category works
print("\nQuick verification for first category:")
first_category = list(QUESTION_CATEGORIES.keys())[0]
first_question = QUESTION_CATEGORIES[first_category][0]
orig_col = CLEANED_TO_ORIGINAL_COL.get(first_question)
if orig_col:
    value = first_client.get(orig_col)
    print(f"  ✓ First question '{first_question}' → '{orig_col[:50]}...' → value: {value}")
else:
    print(f"  ✗ First question '{first_question}' → NOT FOUND")


✓ All 24 cleaned question names found in mapping
Testing on first client:
Email: louise.trelles-tvede@indigital.marketing

Category Scores:
  Tech & Data: 1.20
  Campaigning & Assets: 1.33
  Segmentation & Personalisation: 1.33
  Reporting & Insights: 1.00
  People & Operations: 1.50

Quick verification for first category:
  ✓ First question 'how_would_you_rank_the_capabilities_of_your_existing_crm_technology_stack' → 'How would you rank the capabilities of your existi...' → value: 2


In [65]:
# Calculate category scores for all clients
category_scores_list = []

for idx, row in df.iterrows():
    client_scores = calculate_category_scores(row, QUESTION_CATEGORIES, CLEANED_TO_ORIGINAL_COL)
    client_scores['Email Address'] = row['Email Address']
    client_scores['Timestamp'] = row['Timestamp']
    category_scores_list.append(client_scores)

# Create a dataframe with category scores
scores_df = pd.DataFrame(category_scores_list)

# Reorder columns to have Email and Timestamp first
cols = ['Email Address', 'Timestamp'] + [c for c in scores_df.columns if c not in ['Email Address', 'Timestamp']]
scores_df = scores_df[cols]

print(f"✓ Calculated category scores for {len(scores_df)} clients")
print(f"\nCategory Scores DataFrame:")
scores_df


✓ Calculated category scores for 3 clients

Category Scores DataFrame:


,Email Address,Timestamp,Tech & Data,Campaigning & Assets,Segmentation & Personalisation,Reporting & Insights,People & Operations
0,louise.trelles-tvede@indigital.marketing,11/2/2025 17:13:08,1.2,1.333333,1.333333,1.000000,1.50
1,louisettvede@gmail.com,11/4/2025 14:48:53,2.8,2.166667,2.666667,1.333333,1.50
2,shahruzahmed1995@gmail.com,11/7/2025 10:00:23,2.8,3.000000,2.666667,2.500000,2.25


In [66]:
# Display summary statistics
print("Summary Statistics by Category:")
print(scores_df[[c for c in scores_df.columns if c not in ['Email Address', 'Timestamp']]].describe())


Summary Statistics by Category:
       Tech & Data  Campaigning & Assets  Segmentation & Personalisation  \
count     3.000000              3.000000                        3.000000   
mean      2.266667              2.166667                        2.222222   
std       0.923760              0.833333                        0.769800   
min       1.200000              1.333333                        1.333333   
25%       2.000000              1.750000                        2.000000   
50%       2.800000              2.166667                        2.666667   
75%       2.800000              2.583333                        2.666667   
max       2.800000              3.000000                        2.666667   

       Reporting & Insights  People & Operations  
count              3.000000             3.000000  
mean               1.611111             1.750000  
std                0.787636             0.433013  
min                1.000000             1.500000  
25%                1.166667 

## Examine PowerPoint Template

Let's explore the structure of the PowerPoint template to understand how to populate it.


In [67]:
from pptx import Presentation

# Load the PowerPoint template
template_path = 'Maturity_Slide_Template.pptx'
prs = Presentation(template_path)

print(f"✓ Loaded PowerPoint template: {template_path}")
print(f"Total slides: {len(prs.slides)}")
print(f"\nSlide breakdown:")
print("="*60)

for i, slide in enumerate(prs.slides, 1):
    print(f"\nSlide {i}:")
    # Try to get slide title and show shapes
    title = "No title found"
    for shape in slide.shapes:
        if hasattr(shape, "text") and shape.text.strip():
            text_preview = shape.text.strip()[:80]
            if len(shape.text.strip()) > 80:
                text_preview += "..."
            print(f"  Shape: {shape.shape_type} - Text preview: '{text_preview}'")
            # Check if it might be a title (first non-empty text or placeholder)
            if title == "No title found":
                title = shape.text.strip()[:50]
                if len(shape.text.strip()) > 50:
                    title += "..."
    
    # Count shapes
    print(f"  Total shapes: {len(slide.shapes)}")
    print(f"  Placeholders: {len([s for s in slide.shapes if hasattr(s, 'is_placeholder') and s.is_placeholder])}")


✓ Loaded PowerPoint template: Maturity_Slide_Template.pptx
Total slides: 5

Slide breakdown:

Slide 1:
  Shape: PLACEHOLDER (14) - Text preview: 'Tech and Data'
  Shape: TEXT_BOX (17) - Text preview: 'Your score'
  Shape: TEXT_BOX (17) - Text preview: 'Summary & recommendations'
  Shape: TEXT_BOX (17) - Text preview: 'Lorem ipsum summary'
  Shape: AUTO_SHAPE (1) - Text preview: '✓'
  Shape: AUTO_SHAPE (1) - Text preview: '✓'
  Shape: AUTO_SHAPE (1) - Text preview: '✓'
  Shape: AUTO_SHAPE (1) - Text preview: '✓'
  Shape: TEXT_BOX (17) - Text preview: 'Recommendation 1

Recommendation 2
Recommendation 3
Recommendation 4'
  Total shapes: 12
  Placeholders: 1

Slide 2:
  Shape: PLACEHOLDER (14) - Text preview: 'Campaigning & Assets'
  Shape: TEXT_BOX (17) - Text preview: 'Your score'
  Shape: TEXT_BOX (17) - Text preview: 'Summary & recommendations'
  Shape: TEXT_BOX (17) - Text preview: 'Lorem ipsum summary'
  Shape: AUTO_SHAPE (1) - Text preview: '✓'
  Shape: AUTO_SHAPE (1) - Text previe

In [68]:
# More detailed examination - look at each slide's structure
print("Detailed slide structure:")
print("="*60)

for i, slide in enumerate(prs.slides, 1):
    print(f"\n{'='*60}")
    print(f"SLIDE {i}")
    print(f"{'='*60}")
    
    for j, shape in enumerate(slide.shapes, 1):
        shape_info = f"  Shape {j}: {shape.shape_type}"
        
        # Check if it's a placeholder
        if hasattr(shape, 'is_placeholder') and shape.is_placeholder:
            shape_info += f" [PLACEHOLDER: {shape.placeholder_format.type}]"
        
        # Get text if available
        if hasattr(shape, "text"):
            text = shape.text.strip()
            if text:
                shape_info += f"\n    Text: '{text[:100]}{'...' if len(text) > 100 else ''}'"
        
        print(shape_info)


Detailed slide structure:

SLIDE 1
  Shape 1: PLACEHOLDER (14) [PLACEHOLDER: TITLE (1)]
    Text: 'Tech and Data'
  Shape 2: TABLE (19)
  Shape 3: TEXT_BOX (17)
    Text: 'Your score'
  Shape 4: TEXT_BOX (17)
    Text: 'Summary & recommendations'
  Shape 5: TEXT_BOX (17)
    Text: 'Lorem ipsum summary'
  Shape 6: AUTO_SHAPE (1)
    Text: '✓'
  Shape 7: AUTO_SHAPE (1)
    Text: '✓'
  Shape 8: AUTO_SHAPE (1)
    Text: '✓'
  Shape 9: AUTO_SHAPE (1)
    Text: '✓'
  Shape 10: TEXT_BOX (17)
    Text: 'Recommendation 1

Recommendation 2
Recommendation 3
Recommendation 4'
  Shape 11: LINE (9)
  Shape 12: AUTO_SHAPE (1)

SLIDE 2
  Shape 1: PLACEHOLDER (14) [PLACEHOLDER: TITLE (1)]
    Text: 'Campaigning & Assets'
  Shape 2: TABLE (19)
  Shape 3: TEXT_BOX (17)
    Text: 'Your score'
  Shape 4: TEXT_BOX (17)
    Text: 'Summary & recommendations'
  Shape 5: TEXT_BOX (17)
    Text: 'Lorem ipsum summary'
  Shape 6: AUTO_SHAPE (1)
    Text: '✓'
  Shape 7: AUTO_SHAPE (1)
    Text: '✓'
  Shape 8: AUTO_

## Generate Recommendations & Populate Slides

Now we'll generate recommendations using OpenAI and populate the PowerPoint slides.


In [69]:
# Map slide titles to our category names
SLIDE_CATEGORY_MAPPING = {
    'Tech and Data': 'Tech & Data',
    'Campaigning & Assets': 'Campaigning & Assets',
    'Segmentation & Personalisation': 'Segmentation & Personalisation',
    'Reporting & Insights': 'Reporting & Insights',
    'People & Operations': 'People & Operations'
}

# Reverse mapping: our category -> slide index (0-based)
CATEGORY_TO_SLIDE = {}
for i, slide in enumerate(prs.slides):
    # Get the title from the first placeholder
    for shape in slide.shapes:
        if hasattr(shape, 'is_placeholder') and shape.is_placeholder:
            title = shape.text.strip()
            if title in SLIDE_CATEGORY_MAPPING:
                category = SLIDE_CATEGORY_MAPPING[title]
                CATEGORY_TO_SLIDE[category] = i
                break

print("Category to Slide mapping:")
for category, slide_idx in CATEGORY_TO_SLIDE.items():
    print(f"  {category} → Slide {slide_idx + 1}")


Category to Slide mapping:
  Tech & Data → Slide 1
  Campaigning & Assets → Slide 2
  Segmentation & Personalisation → Slide 3
  Reporting & Insights → Slide 4
  People & Operations → Slide 5


In [70]:
# Function to find text boxes and orange circle on a slide
def find_text_boxes(slide):
    """
    Find the key elements we need to update:
    - Score box (contains "Your score")
    - Recommendations box (contains "Recommendation 1")
    - Orange circle (AUTO_SHAPE that's likely the indicator)
    - Horizontal line (to position circle on)
    
    Note: We only update these elements. Everything else is left unchanged.
    """
    elements = {
        'score': None,
        'recommendations': None,
        'orange_circle': None,
        'line': None
    }
    
    # First pass: find text boxes
    for shape in slide.shapes:
        if hasattr(shape, "text"):
            text = shape.text.strip()
            if "Your score" in text and elements['score'] is None:
                elements['score'] = shape
            elif "Recommendation 1" in text and elements['recommendations'] is None:
                elements['recommendations'] = shape
    
    # Second pass: find orange circle and line
    # Look for LINE shape (the horizontal line)
    lines = []
    circles = []
    
    for shape in slide.shapes:
        # Check for line shape
        if hasattr(shape, 'shape_type'):
            if shape.shape_type == 9:  # LINE shape type
                lines.append(shape)
        
        # Collect all AUTO_SHAPE circles (likely candidates for the orange circle)
        if hasattr(shape, 'shape_type') and shape.shape_type == 1:  # AUTO_SHAPE
            # Check if it's roughly circular (width and height similar)
            if hasattr(shape, 'width') and hasattr(shape, 'height'):
                width = shape.width
                height = shape.height
                # Check if it's roughly circular (within 20% difference)
                if abs(width - height) / max(width, height) < 0.2:
                    # Check if it has a fill (likely the orange circle)
                    try:
                        if hasattr(shape, 'fill') and shape.fill.type == 1:  # Solid fill
                            circles.append(shape)
                    except:
                        pass
    
    # Find the horizontal line (longest line, likely horizontal)
    if lines:
        # Find the longest line (likely the horizontal indicator line)
        elements['line'] = max(lines, key=lambda l: l.width if hasattr(l, 'width') else 0)
    
    # Find the orange circle (small filled circle, likely near the line)
    if circles and elements['line']:
        # Find circle closest to the line (vertically)
        line_y = elements['line'].top + (elements['line'].height / 2)
        elements['orange_circle'] = min(circles, 
                                        key=lambda c: abs((c.top + c.height/2) - line_y))
    elif circles:
        # If no line found, just take the first filled circle
        elements['orange_circle'] = circles[0]
    
    return elements

# Test on first slide
test_slide = prs.slides[0]
elements = find_text_boxes(test_slide)
print("Elements we will update on first slide:")
for key, element in elements.items():
    if element:
        if key == 'line':
            print(f"  {key}: Found - Line shape")
        elif key == 'orange_circle':
            print(f"  {key}: Found - Circle shape")
        else:
            print(f"  {key}: Found - Current text preview: '{element.text.strip()[:50]}...'")
    else:
        print(f"  {key}: NOT FOUND")
print("\nNote: Score, recommendations, and orange circle position will be updated.")


Elements we will update on first slide:
  score: Found - Current text preview: 'Your score...'
  recommendations: Found - Current text preview: 'Recommendation 1

Recommendation 2
Recommendation ...'
  orange_circle: Found - Circle shape
  line: Found - Line shape

Note: Score, recommendations, and orange circle position will be updated.


In [71]:
# Install openai if needed
import os

# ============================================
# SET YOUR OPENAI API KEY HERE
# ============================================
# Option 1: Set it directly in this cell (easiest for testing)
# Uncomment the line below and paste your API key:
os.environ['OPENAI_API_KEY'] = 'your-api-key-here'

# Option 2: Set it as an environment variable (more secure for production)
# In terminal, run: export OPENAI_API_KEY='your-api-key-here'
# Or add to your ~/.zshrc or ~/.bashrc file

# Check if OpenAI API key is set
if 'OPENAI_API_KEY' in os.environ and os.environ['OPENAI_API_KEY'] != 'your-api-key-here':
    print("✓ OpenAI API key found in environment")
    print(f"   Key starts with: {os.environ['OPENAI_API_KEY'][:10]}...")
else:
    print("⚠️  OpenAI API key not found. Please set it:")
    print("   1. Uncomment the line above and paste your key, OR")
    print("   2. Run in terminal: export OPENAI_API_KEY='your-api-key-here'")


✓ OpenAI API key found in environment
   Key starts with: sk-proj-ww...


In [72]:
# Set up OpenAI client
try:
    from openai import OpenAI
    
    api_key = os.environ.get('OPENAI_API_KEY')
    if not api_key or api_key == 'your-api-key-here':
        print("⚠️  OpenAI API key not set. Please set it in the cell above.")
        client = None
    else:
        client = OpenAI(api_key=api_key)
        print("✓ OpenAI client initialized and ready to use")
        print("   Model: gpt-4o-mini (cost-effective)")
except ImportError:
    print("⚠️  OpenAI not installed. Run: pip install openai")
    client = None
except Exception as e:
    print(f"⚠️  Error initializing OpenAI: {e}")
    client = None


✓ OpenAI client initialized and ready to use
   Model: gpt-4o-mini (cost-effective)


In [73]:
# Function to generate recommendations using OpenAI
def generate_recommendations(category, score, questions_in_category, client_responses, original_questions_dict):
    """
    Generate recommendations for a category based on the score and responses
    
    Parameters:
    - category: Category name (e.g., "Tech & Data")
    - score: Average score for this category (1-4)
    - questions_in_category: List of cleaned question names in this category
    - client_responses: Dict of cleaned question -> response value
    - original_questions_dict: Dict mapping cleaned question -> original question text
    
    Returns:
    - summary: Brief summary text
    - recommendations: List of 4 recommendation strings
    """
    if client is None:
        # Fallback if OpenAI not available
        return "Summary not available - OpenAI not configured", [
            "Recommendation 1: Configure OpenAI API key",
            "Recommendation 2: Set OPENAI_API_KEY environment variable",
            "Recommendation 3: Install openai package",
            "Recommendation 4: Check API key permissions"
        ]
    
    # Identify low-scoring questions (scores 1-2) that need focus
    low_scoring_questions = []
    all_questions_context = []
    
    for cleaned_q in questions_in_category:
        q_score = client_responses.get(cleaned_q, None)
        if q_score is not None and isinstance(q_score, (int, float)):
            original_q = original_questions_dict.get(cleaned_q, cleaned_q)
            question_info = {
                'cleaned': cleaned_q,
                'original': original_q,
                'score': q_score
            }
            all_questions_context.append(question_info)
            
            # Focus on questions with scores 1-2
            if q_score <= 2:
                low_scoring_questions.append(question_info)
    
    # Sort by score (lowest first) to prioritize worst areas
    low_scoring_questions.sort(key=lambda x: x['score'])
    all_questions_context.sort(key=lambda x: x['score'])
    
    # Build detailed context with actual question text
    questions_detail = "\n".join([
        f"- Question: {q['original']}\n  Score: {q['score']}/4"
        for q in all_questions_context
    ])
    
    # Build focus areas (low-scoring questions)
    focus_areas = ""
    if low_scoring_questions:
        focus_areas = "\n\nAreas requiring immediate attention (low scores):\n" + "\n".join([
            f"- {q['original']} (Score: {q['score']}/4)"
            for q in low_scoring_questions[:3]  # Top 3 lowest scores
        ])
    
    # Determine maturity level
    if score <= 1.5:
        maturity_level = "not mature"
    elif score <= 2.5:
        maturity_level = "developing"
    elif score <= 3.5:
        maturity_level = "mature"
    else:
        maturity_level = "very mature"
    
    # Build the prompt with focus on low-scoring areas
    prompt = f"""You are a CRM marketing maturity consultant. Generate recommendations for a client.

Category: {category}
Overall Maturity Score: {score:.2f}/4.0 ({maturity_level})

Client's responses to all questions in this category:
{questions_detail}
{focus_areas}

Instructions:
1. Generate a brief 2-3 sentence summary of their current maturity level in this category
2. Generate four specific, actionable recommendations that:
   - PRIORITIZE addressing the low-scoring areas identified above
   - Reference the specific questions where they scored low (1-2 out of 4)
   - Provide concrete, actionable steps based on the question context
   - Are tailored to their current maturity level

Focus especially on the questions where they scored 1-2, as these are the areas needing the most improvement.

Format the response as:
SUMMARY: [your summary here]
RECOMMENDATIONS:
1. [recommendation 1 - should address a specific low-scoring question]
2. [recommendation 2 - should address a specific low-scoring question]
3. [recommendation 3 - can address another area or build on improvements]
4. [recommendation 4 - can address another area or build on improvements]

Make each recommendation specific, actionable, and directly related to the questions they answered poorly."""

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # Using cost-effective model
            messages=[
                {"role": "system", "content": "You are an expert CRM marketing consultant providing actionable recommendations."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.7,
            max_tokens=500
        )
        
        result = response.choices[0].message.content
        
        # Parse the response
        if "SUMMARY:" in result:
            parts = result.split("RECOMMENDATIONS:")
            summary = parts[0].replace("SUMMARY:", "").strip()
            recommendations_text = parts[1] if len(parts) > 1 else ""
            
            # Extract recommendations (numbered list)
            recommendations = []
            for line in recommendations_text.split('\n'):
                line = line.strip()
                if line and (line[0].isdigit() or line.startswith('-')):
                    # Remove numbering/bullets
                    rec = line.split('.', 1)[-1].strip() if '.' in line else line.lstrip('- ').strip()
                    if rec:
                        recommendations.append(rec)
            
            # Ensure we have 4 recommendations
            while len(recommendations) < 4:
                recommendations.append("Continue building on the recommendations above.")
            
            return summary[:200], recommendations[:4]
        else:
            # Fallback parsing
            lines = result.split('\n')
            summary = lines[0][:200] if lines else "Summary generated"
            recommendations = [line.strip() for line in lines[1:] if line.strip() and len(line.strip()) > 10][:4]
            while len(recommendations) < 4:
                recommendations.append("Continue improving in this area.")
            return summary, recommendations
            
    except Exception as e:
        print(f"Error generating recommendations: {e}")
        return f"Error: {str(e)}", [
            "Recommendation 1: Review current processes",
            "Recommendation 2: Identify improvement areas",
            "Recommendation 3: Implement best practices",
            "Recommendation 4: Monitor progress"
        ]

# Test the function (commented out until API key is set)
# test_summary, test_recs = generate_recommendations(
#     "Tech & Data", 
#     2.3, 
#     ["Question 1", "Question 2"],
#     {"Question 1": 2, "Question 2": 3}
# )
# print("Test output:")
# print(f"Summary: {test_summary}")
# print(f"Recommendations: {test_recs}")


In [74]:
# Main function to generate PowerPoint for a client
def generate_client_presentation(client_email, client_scores, client_responses, output_filename):
    """
    Generate a personalized PowerPoint presentation for a client
    
    Parameters:
    - client_email: Email address of the client
    - client_scores: Dict of category -> score (from scores_df)
    - client_responses: Dict of original question -> response value
    - output_filename: Name for the output PowerPoint file
    """
    # Load a fresh copy of the template
    prs = Presentation(template_path)
    
    # Get client's question responses (original column names -> values)
    # We need to map cleaned question names back to original for lookup
    question_responses = {}
    for orig_col, cleaned_col in COLUMN_NAME_MAPPING.items():
        if orig_col in client_responses:
            question_responses[cleaned_col] = client_responses[orig_col]
    
    # Process each category
    for category, score in client_scores.items():
        if category not in CATEGORY_TO_SLIDE:
            print(f"⚠️  Warning: Category '{category}' not found in slides")
            continue
        
        if pd.isna(score) or score is None:
            print(f"⚠️  Warning: No score for category '{category}'")
            continue
        
        slide_idx = CATEGORY_TO_SLIDE[category]
        slide = prs.slides[slide_idx]
        
        # Find text boxes and table
        elements = find_text_boxes(slide)
        
        # Get questions for this category
        questions_in_category = QUESTION_CATEGORIES.get(category, [])
        
        # Get responses for questions in this category
        category_responses = {q: question_responses.get(q, 'N/A') for q in questions_in_category}
        
        # Create mapping of cleaned question -> original question text for context
        original_questions_dict = {}
        for cleaned_q in questions_in_category:
            orig_col = CLEANED_TO_ORIGINAL_COL.get(cleaned_q)
            if orig_col:
                original_questions_dict[cleaned_q] = orig_col
        
        # Generate recommendations (now with question context and focus on low scores)
        print(f"Generating recommendations for {category} (score: {score:.2f})...")
        summary, recommendations = generate_recommendations(
            category, 
            score, 
            questions_in_category,
            category_responses,
            original_questions_dict  # Pass original question text for context
        )
        
        # Only update "Your score" text box
        if elements['score']:
            # Find the text frame and update it
            if hasattr(elements['score'], 'text_frame'):
                # Clear existing text
                elements['score'].text_frame.clear()
                # Add new text
                p = elements['score'].text_frame.paragraphs[0]
                p.text = f"{score:.2f}/4.0"
            else:
                elements['score'].text = f"{score:.2f}/4.0"
        
        # Only update recommendations text box (leave summary and table unchanged)
        if elements['recommendations']:
            recommendations_text = "\n\n".join([f"{i+1}. {rec}" for i, rec in enumerate(recommendations)])
            if hasattr(elements['recommendations'], 'text_frame'):
                elements['recommendations'].text_frame.clear()
                p = elements['recommendations'].text_frame.paragraphs[0]
                p.text = recommendations_text
            else:
                elements['recommendations'].text = recommendations_text
        
        # Position orange circle based on score
        # Score ranges: 0-1 (FOUNDATIONAL), 1-2 (DEVELOPING), 2-3 (ADVANCED), 3-4 (LEADING)
        if elements['orange_circle'] and elements['line']:
            try:
                # Get line position and dimensions
                line = elements['line']
                line_left = line.left
                line_width = line.width
                line_top = line.top
                line_height = line.height
                
                # Calculate circle position based on score (0-4 scale)
                # Map score to position along the line
                # 0 = left edge, 4 = right edge
                score_position = score / 4.0  # Convert to 0-1 range
                circle_x = line_left + (line_width * score_position)
                
                # Center circle vertically on the line
                circle_y = line_top + (line_height / 2)
                
                # Get circle dimensions to center it properly
                circle = elements['orange_circle']
                circle_width = circle.width
                circle_height = circle.height
                
                # Position circle center on the line
                circle.left = int(circle_x - (circle_width / 2))
                circle.top = int(circle_y - (circle_height / 2))
                
                print(f"  ✓ Positioned orange circle at score {score:.2f} (position: {score_position:.1%} along line)")
            except Exception as e:
                print(f"  ⚠️  Could not position orange circle: {e}")
    
    # Save the presentation
    prs.save(output_filename)
    print(f"\n✓ Presentation saved: {output_filename}")
    return output_filename

print("✓ Function ready to generate client presentations")


✓ Function ready to generate client presentations


In [75]:
# Generate presentations for all clients
# First, let's prepare the data we need

# Create a function to get all responses for a client
def get_client_responses(client_row):
    """Get all question responses for a client as a dict"""
    responses = {}
    question_columns = [col for col in df.columns if col not in ['Timestamp', 'Email Address']]
    for col in question_columns:
        responses[col] = client_row[col]
    return responses

# Helper function to create safe filename from email
def email_to_filename(email, prefix="", suffix="Maturity_Assessment"):
    """
    Convert email address to safe filename with email as prefix
    
    Parameters:
    - email: Email address
    - prefix: Optional prefix (default: empty)
    - suffix: Suffix for filename (default: "Maturity_Assessment")
    
    Returns:
    - Safe filename string
    """
    # Replace @ with _at_ and . with _ for safety
    safe_email = email.replace('@', '_at_').replace('.', '_')
    
    # Build filename: email_prefix_suffix.pptx
    if prefix:
        filename = f"{safe_email}_{prefix}_{suffix}.pptx"
    else:
        filename = f"{safe_email}_{suffix}.pptx"
    
    return filename

# Test with first client
print("Testing with first client:")
first_client_email = scores_df.iloc[0]['Email Address']
first_client_scores = {cat: scores_df.iloc[0][cat] for cat in QUESTION_CATEGORIES.keys()}
first_client_row = df[df['Email Address'] == first_client_email].iloc[0]
first_client_responses = get_client_responses(first_client_row)

print(f"Client: {first_client_email}")
print(f"Scores: {first_client_scores}")
print(f"Filename would be: {email_to_filename(first_client_email)}")

# Uncomment to generate (requires OpenAI API key):
# output_file = email_to_filename(first_client_email)
# generate_client_presentation(first_client_email, first_client_scores, first_client_responses, output_file)


Testing with first client:
Client: louise.trelles-tvede@indigital.marketing
Scores: {'Tech & Data': np.float64(1.2), 'Campaigning & Assets': np.float64(1.3333333333333333), 'Segmentation & Personalisation': np.float64(1.3333333333333333), 'Reporting & Insights': np.float64(1.0), 'People & Operations': np.float64(1.5)}
Filename would be: louise_trelles-tvede_at_indigital_marketing_Maturity_Assessment.pptx


## Generate Presentations for All Clients

Generate personalized PowerPoint presentations for each client.


In [76]:
# Generate presentations for all clients
def generate_all_presentations(scores_df, df, output_dir="output"):
    """
    Generate PowerPoint presentations for all clients
    
    Parameters:
    - scores_df: DataFrame with category scores per client
    - df: Original dataframe with all responses
    - output_dir: Directory to save presentations (default: "output")
    """
    import os
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    generated_files = []
    
    for idx, row in scores_df.iterrows():
        client_email = row['Email Address']
        
        # Get scores for this client
        client_scores = {cat: row[cat] for cat in QUESTION_CATEGORIES.keys()}
        
        # Get original responses
        client_row = df[df['Email Address'] == client_email].iloc[0]
        client_responses = get_client_responses(client_row)
        
        # Create filename with email as prefix
        filename = email_to_filename(client_email)
        output_filename = os.path.join(output_dir, filename)
        
        print(f"\n{'='*60}")
        print(f"Processing: {client_email}")
        print(f"Output file: {filename}")
        print(f"{'='*60}")
        
        try:
            generate_client_presentation(
                client_email, 
                client_scores, 
                client_responses, 
                output_filename
            )
            generated_files.append(output_filename)
        except Exception as e:
            print(f"❌ Error generating presentation for {client_email}: {e}")
    
    print(f"\n{'='*60}")
    print(f"✓ Generated {len(generated_files)} presentations")
    print(f"Saved to: {output_dir}/")
    print(f"\nFiles generated:")
    for f in generated_files:
        print(f"  - {os.path.basename(f)}")
    return generated_files

# Uncomment to generate all presentations (requires OpenAI API key):
generated = generate_all_presentations(scores_df, df)



Processing: louise.trelles-tvede@indigital.marketing
Output file: louise_trelles-tvede_at_indigital_marketing_Maturity_Assessment.pptx
Generating recommendations for Tech & Data (score: 1.20)...
  ✓ Positioned orange circle at score 1.20 (position: 30.0% along line)
Generating recommendations for Campaigning & Assets (score: 1.33)...
  ✓ Positioned orange circle at score 1.33 (position: 33.3% along line)
Generating recommendations for Segmentation & Personalisation (score: 1.33)...
  ✓ Positioned orange circle at score 1.33 (position: 33.3% along line)
Generating recommendations for Reporting & Insights (score: 1.00)...
  ✓ Positioned orange circle at score 1.00 (position: 25.0% along line)
Generating recommendations for People & Operations (score: 1.50)...
  ✓ Positioned orange circle at score 1.50 (position: 37.5% along line)

✓ Presentation saved: output/louise_trelles-tvede_at_indigital_marketing_Maturity_Assessment.pptx

Processing: louisettvede@gmail.com
Output file: louisettved